In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols

import geopandas as gpd

output_dir = '../data/curated/'


In [8]:
properties_df = pd.read_csv("../data/curated/properties_processed.csv", index_col=0)
properties_df['Postcode'] = properties_df['Postcode'].astype(str)

census_df = pd.read_csv("../data/curated/census_data.csv")

sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_map.csv", index_col=0)
sa2_postcode_map['SA2_MAINCODE_2011'] = sa2_postcode_map['SA2_MAINCODE_2011'].astype(int)

In [31]:
census_df[census_df['SA2_CODE_2021'] == 206051511]

,SA2_CODE_2021,Tot_persons_C11_P,Tot_persons_C16_P,Tot_persons_C21_P,Med_mortg_rep_mon_C2011,Med_mortg_rep_mon_C2016,Med_mortg_rep_mon_C2021,Med_person_inc_we_C2011,Med_person_inc_we_C2016,Med_person_inc_we_C2021,Med_rent_weekly_C2011,Med_rent_weekly_C2016,Med_rent_weekly_C2021,Med_tot_hh_inc_wee_C2011,Med_tot_hh_inc_wee_C2016,Med_tot_hh_inc_wee_C2021,Average_hh_size_C2011,Average_hh_size_C2016,Average_hh_size_C2021
147,206051511,91,94,1907,2773,2700,2279,1010,1361,1346,514,500,585,1666,1791,2407,1.8,1.7,2.1


In [9]:
median_rent = properties_df.groupby(["Postcode"])["Cost"].median()


,SA2_CODE_2021,Tot_persons_C11_P,Tot_persons_C16_P,Tot_persons_C21_P,Med_mortg_rep_mon_C2011,Med_mortg_rep_mon_C2016,Med_mortg_rep_mon_C2021,Med_person_inc_we_C2011,Med_person_inc_we_C2016,Med_person_inc_we_C2021,Med_rent_weekly_C2011,Med_rent_weekly_C2016,Med_rent_weekly_C2021,Med_tot_hh_inc_wee_C2011,Med_tot_hh_inc_wee_C2016,Med_tot_hh_inc_wee_C2021,Average_hh_size_C2011,Average_hh_size_C2016,Average_hh_size_C2021
0,201011001,8348,11658,16835,1565,1615,1698,607,702,865,250,311,370,1406,1585,1952,2.8,2.8,2.8
1,201011002,12076,12046,12131,1430,1500,1700,575,670,842,230,260,313,1120,1327,1573,2.3,2.3,2.2
2,201011005,6604,7153,7261,1517,1580,1662,532,638,805,260,300,330,1401,1634,1927,2.8,2.8,2.7
3,201011006,5749,7082,10661,1430,1466,1500,514,595,775,195,265,360,1073,1272,1627,2.7,2.6,2.6
4,201011007,3782,3935,4230,1558,1600,1733,565,646,802,240,260,350,1336,1583,2065,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,217041478,8888,9433,10014,1300,1400,1517,522,617,758,200,250,280,1025,1217,1522,2.5,2.5,2.4
520,217041479,19542,21101,22405,1504,1500,1451,536,625,760,226,255,300,1047,1210,1451,2.5,2.5,2.4
521,217041480,12842,12919,13337,1452,1364,1387,526,610,761,230,250,287,997,1144,1380,2.3,2.3,2.2
522,297979799,38,26,38,0,0,0,1949,9954,2200,0,0,0,0,0,0,0.0,0.0,0.0


In [14]:
median_rent.to_csv(f'{output_dir}median_rental_postcode.csv')